In [1]:
import sys
!{sys.executable} -m pip install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!{sys.executable} -m pip install torch -q
!{sys.executable} -m pip install torchvision -q
!{sys.executable} -m pip install pytorch_lightning -q
!{sys.executable} -m pip install pytorch-lightning -q

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


In [2]:
import pandas as pd
import numpy as np
from PIL import Image
import math

import torch
import torch.nn as nn
import torchmetrics
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms


import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
#from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping


import pickle

In [3]:


#available columns
with open('masterdata/tab_data_cols.pkl', 'rb') as f:
        all_cols = pickle.load(f)


In [4]:


#actually used columns
used_cols = all_cols
used_cols.remove('match_id')
#used_cols.remove('time')
used_cols.remove('winner')
used_cols.remove('duration')
used_cols.remove('p1_civ')
used_cols.remove('p2_civ')
used_cols.remove('map')
used_cols.remove('map_size')
used_cols.remove('imgf_Move_layer')



#value to predict
pred_col = 'winner'

#img input layers
#img_layer_names = ['gaia_layer','Build_layer','Move_layer','Target_layer']
img_layer_names= ['Build_layer']


img_channels_count = int(len(img_layer_names)*3)


#early stop
early_stop_callback = False

In [5]:
len(used_cols)
used_cols_bu = used_cols.copy()
p1_cols = []

tbd_list = []
#delete opp info
for col in used_cols:
    if 'p2' not in col:
        p1_cols.append(col)
        
print(len(p1_cols))
len(used_cols)

335


669

In [6]:
acc_dict = {}
time_list = [t for t in range(0,5001,120)]
elo_list = ['all','low','medium','high']
scenarios = [1,2]
global size_dict
size_dict = {}

In [ ]:
    
for scenario in scenarios:
    
    if scenario == 1:
        used_cols = p1_cols.copy()
    elif scenario ==2: 
        used_cols = used_cols_bu.copy()
    col_count = len(used_cols)
    
    acc_dict[scenario] = {}

    for elo in elo_list:
        #define parameters
        acc_dict[scenario][elo] = {}

        data_path = f"./data/minn/elo_{elo}/"
        for tf in time_list:
            #create custom PyTorch ImageDataset class
            val_acc_values = []
            val_acc_values_end = []
            test_acc_values = []
            


            class ImageDataset(Dataset):
                """Tabular and Image dataset."""

                def __init__(self, pickle_file, image_dir):
                    self.image_dir = image_dir
                    self.pickle_file = pickle_file
                    self.tabular = pd.read_pickle(pickle_file)

                def __len__(self):
                    return len(self.tabular)

                def __getitem__(self, idx):
                    if torch.is_tensor(idx):
                        idx = idx.tolist()

                    tabular = self.tabular.iloc[idx, 0:]

                    y = tabular[pred_col]

                    image = 'dummy'
                    #image = Image.open(f"{self.image_dir}{tabular['imgf_Build_layer']}")
                    #image = np.array(image)
                    #image = image[..., :3]

                    #print(image.shape)

                    #image = transforms.functional.to_tensor(image)

                    tabular = tabular[used_cols]
                    tabular = tabular.tolist()
                    tabular = torch.FloatTensor(tabular)

                    #print(y,tabular)

                    return image, tabular, y
            #Create Architecture
            ##create custom cnn layer
            def conv_block(input_size, output_size):
                block = nn.Sequential(
                    nn.Conv2d(input_size, output_size, (3, 3)), nn.ReLU(), nn.BatchNorm2d(output_size), nn.MaxPool2d((2, 2)),
                )

                return block


            ##define layers used in network
            class LitClassifier(pl.LightningModule):
                def __init__(
                    self, lr: float = 1e-3, num_workers: int = 4, batch_size: int = 32,
                ):
                    super().__init__()
                    self.lr = lr
                    self.num_workers = num_workers
                    self.batch_size = batch_size

                    self.conv1 = conv_block(img_channels_count, 16)
                    self.conv2 = conv_block(16, 32)
                    self.conv3 = conv_block(32, 64)

                    #self.ln1 = nn.Linear(10816, 16)
                    self.ln1 = nn.Linear(64 * 13 * 13, 16)
                    self.relu = nn.ReLU()
                    self.batchnorm = nn.BatchNorm1d(16)
                    self.dropout = nn.Dropout2d(0.5)
                    self.ln2 = nn.Linear(16, 5)

                    self.ln4 = nn.Linear(col_count, 10)
                    self.ln5 = nn.Linear(10, 10)
                    self.ln6 = nn.Linear(10, 5)
                    self.ln7 = nn.Linear(10, 1)

                    self.softmax = nn.Softmax(dim=1)

                    ##representation of architecture with 3 conv layers for image data and multiplay-perceptron for tablular data 
                #- in the end output of CNN and MLP are combinde
                def forward(self, img, tab):
                    #print("Initial: ",img.shape)
                    #img = self.conv1(img)
                    #print("Conv1: ",img.shape)
                    #img = self.conv2(img)
                    #print("Conv2: ",img.shape)
                    #img = self.conv3(img)
                    #print("Conv3: ",img.shape)
                    #img = img.reshape(img.shape[0], -1)
                    #print("After Reshape:",img.shape)
                    #img = self.ln1(img)
                    #print("Ln1: ",img.shape)
                    #img = self.relu(img)
                    #img = self.batchnorm(img)
                    #img = self.dropout(img)
                    #img = self.ln2(img)
                    #img = self.relu(img)
                    #img = self.softmax(img)

                    tab = self.ln4(tab)
                    tab = self.relu(tab)
                    tab = self.ln5(tab)
                    tab = self.relu(tab)
                    tab = self.ln6(tab)
                    tab = self.relu(tab)
                    tab = self.softmax(tab)

                    x = torch.cat((tab, tab), dim=1)
                    #x = torch.cat((tab,tab), dim=1)
                    x = self.relu(x)
                    x = self.ln7(x)

                    return x#self.softmax(x)#self.ln7(x)
                    #add softmax function after last linear layer

                #define training validation & test steps
                def training_step(self, batch, batch_idx):
                    image, tabular, y = batch

                    criterion = torch.nn.L1Loss()
                    y_pred = torch.flatten(self(image, tabular))
                    y_pred = y_pred.double()

                    loss = criterion(y_pred, y)
                    acc = torchmetrics.functional.accuracy(y_pred, y)

                    #print("Train_acc: ",acc)

                    tensorboard_logs = {"train_loss": loss, "train_acc": acc}
                    return {"loss": loss, "log": tensorboard_logs, "train_acc": acc}

                def validation_step(self, batch, batch_idx):
                    image, tabular, y = batch

                    criterion = torch.nn.L1Loss()
                    y_pred = torch.flatten(self(image, tabular))
                    y_pred = y_pred.double()

                    val_loss = criterion(y_pred, y)
                    val_acc = torchmetrics.functional.accuracy(y_pred, y)
                    val_acc_values.append(val_acc.item())

                    #print("Val_acc: ",val_acc)

                    return {"val_loss": val_loss, "val_acc": val_acc}

                def validation_epoch_end(self, outputs):
                    avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
                    avg_acc = torch.stack([x["val_acc"] for x in outputs]).mean()

                    val_acc_values_end.append(avg_acc.item())

                    #print("avg_acc: ", avg_acc)


                    tensorboard_logs = {"val_loss": avg_loss, "avg_acc": avg_acc}
                    return {"val_loss": avg_loss, "log": tensorboard_logs, "avg_acc": avg_acc}

                def test_step(self, batch, batch_idx):
                    image, tabular, y = batch

                    criterion = torch.nn.L1Loss()
                    y_pred = torch.flatten(self(image, tabular))
                    y_pred = y_pred.double()

                    test_loss = criterion(y_pred, y)
                    test_acc =  torchmetrics.functional.accuracy(y_pred, y)

                    test_acc_values.append(test_acc.item())

                    return {"test_loss": test_loss, "test_acc": test_acc}

                def test_epoch_end(self, outputs):
                    avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
                    avg_acc = torch.stack([x["test_acc"] for x in outputs]).mean()

                    logs = {"test_loss": avg_loss, "test_acc": avg_acc}


                    #print("Test_Acc: ", avg_acc)
                    return {"test_loss": avg_loss, "test_acc": avg_acc, "log": logs, "progress_bar": logs}


                #load dataset and define train/test splio

                ##Please note that this code will only work if 0.8*your data size will return whole numbers.
                def setup(self, stage):

                    image_data = ImageDataset(pickle_file=f"{data_path}recordfile_tab_data_{tf}.pkl", image_dir=f"{data_path}")
                    
                    size_dict[f'{scenario}-{elo}-{tf}'] = len(image_data)
                    len_image_data = len(image_data)
                    train_size = math.floor(len_image_data*0.8) + (len_image_data%5)
                    train_size += train_size%2
                    
                    val_size = int((len_image_data - train_size)  / 2)
                    test_size = int((len_image_data - train_size) / 2)

                    if (len_image_data - train_size)%2 != 0:
                        train_size += 1

                    print("Total Size: ", len_image_data,"Train Size: ", train_size, "Val Size: ",val_size, "Test Size: ",test_size)


                    self.train_set, self.val_set, self.test_set = random_split(image_data, (train_size, val_size, test_size),generator=torch.Generator().manual_seed(1))
                    print("Len Train Set: ", len(self.train_set))

                def configure_optimizers(self):
                    return torch.optim.Adam(self.parameters(), lr=(self.lr))

                def train_dataloader(self):
                    return DataLoader(self.train_set, batch_size=self.batch_size, num_workers = 8)

                def val_dataloader(self):
                    return DataLoader(self.val_set, batch_size=self.batch_size, num_workers = 8)

                def test_dataloader(self):
                    return DataLoader(self.test_set, batch_size=self.batch_size, num_workers = 8)

            if __name__ == "__main__":
                logger = TensorBoardLogger("lightning_logs", name="multi_input")
                #early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=5000, patience=7, verbose=False, mode="min")
            
            import warnings
            warnings.filterwarnings("ignore", category=DeprecationWarning) 

            model = LitClassifier()
            #trainer = pl.Trainer(gpus=1, logger=logger, auto_lr_find=True) 
            trainer = pl.Trainer(gpus=1, logger=logger, max_epochs = 9, auto_lr_find=True, log_every_n_steps=1)

            #fit and test
            trainer.fit(model)
            trainer.test(model)


           
            acc_dict[scenario][elo][tf] = sum(test_acc_values)/len(test_acc_values)
            
            output = open(f'data/minn/logs/MINN/acc_dict.pkl' , 'wb')
            pickle.dump(acc_dict, output)
            
            output = open(f'data/minn/logs/MINN/size_dict.pkl' , 'wb')
            pickle.dump(size_dict, output)
            #output = open(f'data/minn/logs/MINN/scenario_{scenario}_elo_{elo}_tf_{tf}_acc_dict.pkl' , 'wb')
            #pickle.dump(acc_dict[scenario], output)
            print(f'SCEN {scenario} ELO {elo} TF {tf} ACC {acc_dict[scenario][elo][tf]}')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Total Size:  138902 Train Size:  111124 Val Size:  13889 Test Size:  13889
Len Train Set:  111124



   | Name      | Type        | Params
-------------------------------------------
0  | conv1     | Sequential  | 480   
1  | conv2     | Sequential  | 4.7 K 
2  | conv3     | Sequential  | 18.6 K
3  | ln1       | Linear      | 173 K 
4  | relu      | ReLU        | 0     
5  | batchnorm | BatchNorm1d | 32    
6  | dropout   | Dropout2d   | 0     
7  | ln2       | Linear      | 85    
8  | ln4       | Linear      | 3.4 K 
9  | ln5       | Linear      | 110   
10 | ln6       | Linear      | 55    
11 | ln7       | Linear      | 11    
12 | softmax   | Softmax     | 0     
-------------------------------------------
200 K     Trainable params
0         Non-trainable params
200 K     Total params
0.802     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
acc_dict

In [ ]:
size_dict
